In [1]:
import pandas as pd 
import numpy as np
import nltk 
from nltk.corpus import stopwords
from nltk.corpus import words
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import scipy.sparse
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sibongiletoure/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Write recommender function that will go into application

In [2]:
music_df = pd.read_csv('Flask_app/data/music_recommender.csv')

In [7]:
def mood_recommendations(df,title_id,valence,energy):
    ''' 
    mood_recommendations takes in 3 things in user input the energy valence and song. 
    Valence - Measures the happiness of a song will return song within 0.05 in either direction
    Energy- Measures how energetic the song is will return songs within 0.05 in either direction
    Title_Id - Will be linked to the song entered and will return any song under the same topic
    '''
    topic = df[df['id']==title_id]['dominant_topic']
    print(topic.item())
    genre = df[df['id']==title_id]['genre']
    genres=[]
    nongenre = []
    
    df = df[df['valence'] >= valence-0.05]
    df = df[df['valence'] <= valence+0.05]
    df = df[df['energy'] >= energy-0.05]
    df = df[df['energy'] <= energy+0.05]
    df = df[df['dominant_topic'] == topic.item()]
    df.reset_index(drop=True,inplace=True)
    for ix,g in enumerate(df.genre):
        if g == genre.item():
            genres.append(ix)
        else:
            nongenre.append(ix)
    ## Put emphasis on songs with same genre but return others as well
    songs_in_genre = list(df.loc[genres]['name'])
    artist_in_genre = list(df.loc[genres]['primary_artist'])
    #audio_in_genre = list(df.loc[genres]['audio_preview'])
    ids_in_genre = list(df.loc[genres]['id'])

    songs_non_genre = list(df.loc[nongenre]['name'])
    artist_non_genre = list(df.loc[nongenre]['primary_artist'])
    #audio_in_genre = list(df.loc[nongenre]['audio_preview'])
    ids_non_genre = list(df.loc[nongenre]['id'])
   
    if len(genres) >= 15:
        return df.loc[genres].head(10)
    elif len(genres) > 0 and len(nongenre)>0:
        return df.loc[nongenres].head(10)
    elif len(genres) > 0 :
        return df.loc[genres].head(10)
    else:
        return df.loc[nongenres].head(10)

## Test to see what is being returned

In [6]:
music_df.iloc[12]

Unnamed: 0                                                         12
acousticness                                                    0.908
artists                                            ['Colbie Caillat']
danceability                                                    0.647
duration_ms                                                    216720
energy                                                         0.0985
explicit                                                            0
id                                             5RMjO7UAJUybWAwZ4t2wKz
instrumentalness                                                    0
key                                                                 9
liveness                                                        0.098
loudness                                                      -12.071
mode                                                                1
name                                        Bubbly - Acoustic Version
popularity          

In [8]:
mood_recommendations(music_df,'5RMjO7UAJUybWAwZ4t2wKz',0.5,0.5)

topic_2


,Unnamed: 0,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,...,speechiness,tempo,valence,year,lyrics,primary_artist,genres,dominant_topic,genre,audio_preview
2,222,0.891000,['The xx'],0.755,177093,0.482,0,4vU90MimdQpV2e1wPi1ljJ,0.818000,0,...,0.0300,109.354,0.481,2009,"You, you used to have all the answers And you,...",The xx,"['downtempo', 'dream pop', 'indietronica']",topic_2,Pop,https://p.scdn.co/mp3-preview/5ca434a4e0b66ccc...
4,369,0.441000,['Indigo Girls'],0.539,216253,0.460,0,4xy8oQb0lmfsK8QhAKkAkl,0.000471,9,...,0.0255,83.904,0.499,1986,I can tell by your eyes That you've probably b...,Indigo Girls,"['folk', 'lilith', 'pop rock']",topic_2,Pop,https://p.scdn.co/mp3-preview/976d925fe84f5178...
7,1164,0.297000,['Halsey'],0.752,201661,0.488,1,5p7ujcrUXASCNwRaWNHR1C,0.000009,6,...,0.0705,136.041,0.533,2018,Found you when your heart was broke I filled y...,Halsey,"['dance pop', 'electropop', 'etherpop', 'indie...",topic_2,Pop,NaN
11,1667,0.291000,['Michael Jackson'],0.651,243853,0.493,0,4NYi8J4w2AuI9qyv5bY2Nt,0.000000,0,...,0.0410,84.966,0.496,1982,Sun comes up on this new morning Shifting shad...,Michael Jackson,"['pop', 'r&b', 'soul']",topic_2,Pop,https://p.scdn.co/mp3-preview/582de3ef1ebc80b6...
12,1681,0.111000,['The Weeknd'],0.582,260253,0.525,0,2Ch7LmS7r2Gy2kc64wv3Bz,0.000000,1,...,0.0627,133.249,0.510,2016,I'm findin' ways to articulate the feeling I'm...,The Weeknd,"['canadian contemporary r&b', 'canadian pop', ...",topic_2,Pop,NaN
13,1962,0.494000,['Brandy'],0.682,218467,0.543,0,5ZCv9I8mgChcYkSrLuLSTc,0.000000,0,...,0.0454,71.582,0.478,1998,"Almost made you love me, almost made you cry A...",Brandy,"['dance pop', 'deep pop r&b', 'hip hop', 'hip ...",topic_2,Pop,https://p.scdn.co/mp3-preview/105cc987313a7187...
14,2051,0.305000,"['Musiq Soulchild', 'Osunlade']",0.554,221240,0.472,0,6dFs9SRuQ8FKK3WORqoa3D,0.000492,0,...,0.2850,141.283,0.497,2000,Maybe my mind is playing tricks on me but I co...,Musiq Soulchild,"['dance pop', 'hip pop', 'neo soul', 'new jack...",topic_2,Pop,NaN
16,2329,0.212000,['Babyface'],0.745,246178,0.451,0,4AFhkn04YASMuvUxVMJKmW,0.000000,11,...,0.1120,143.908,0.481,2001,I ran into a friend of yours the other day And...,Babyface,"['dance pop', 'neo soul', 'new jack swing', 'q...",topic_2,Pop,https://p.scdn.co/mp3-preview/064013c0a02e4a69...
18,2618,0.576000,['The Beach Boys'],0.209,194800,0.515,0,3C0feBlzUt7b3PyPUn7Lfl,0.000000,10,...,0.0353,185.218,0.512,1966,I keep looking for a place to fit Where I can ...,The Beach Boys,"['baroque pop', 'brill building pop', 'classic...",topic_2,Pop,NaN
20,3005,0.000746,['Saint Etienne'],0.593,223933,0.539,0,2caDMtjF01GlxCU7yHUdhu,0.001700,5,...,0.0407,97.920,0.504,1992,"Dry your eyes, boy, there's no need to be sad ...",Saint Etienne,"['art pop', 'big beat', 'britpop', 'chamber ps...",topic_2,Pop,https://p.scdn.co/mp3-preview/596bb8369a5fc26d...


## With an energy and valence of 0.5 the results look pretty good will try with a lower score

In [9]:
mood_recommendations(music_df,'5RMjO7UAJUybWAwZ4t2wKz',0.3,0.3)

topic_2


,Unnamed: 0,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,...,speechiness,tempo,valence,year,lyrics,primary_artist,genres,dominant_topic,genre,audio_preview
2,624,0.8250,"['Ariana Grande', 'Nathan Sykes']",0.324,327773,0.327,0,442j8VxaB60dWf9cBFuX5w,0.000000,2,...,0.0409,81.328,0.334,2013,I'd like to say we gave it a try I'd like to b...,Ariana Grande,"['dance pop', 'pop', 'post-teen pop']",topic_2,Pop,NaN
3,778,0.7960,['ABBA'],0.475,233720,0.260,0,6TvxPS4fj4LUdjw2es4g21,0.001600,5,...,0.0322,137.212,0.339,1981,"Schoolbag in hand, she leaves home in the earl...",ABBA,"['europop', 'swedish pop']",topic_2,Pop,NaN
4,810,0.9200,['The Platters'],0.203,160933,0.295,0,2vSxjnyrWrtxyfzO47EX6q,0.000002,3,...,0.0323,171.222,0.285,1959,They asked me how I knew My true love was true...,The Platters,"['adult standards', 'brill building pop', 'doo...",topic_2,Pop,NaN
8,1331,0.5600,['Bush'],0.648,212893,0.278,0,3ZoAB9gq2RGW5KdHnRvH75,0.007280,11,...,0.0370,118.827,0.303,2005,"It must be your skin, I'm sinking in It must b...",Bush,"['alternative metal', 'alternative rock', 'gru...",topic_2,Pop,https://p.scdn.co/mp3-preview/cedb9c6e986a8449...
14,2904,0.7170,['James Bay'],0.556,275493,0.298,0,0GDQXt7qRJIDDUmcufomrU,0.000017,10,...,0.0344,130.598,0.278,2014,"Home now, end of the night It's colder to turn...",James Bay,"['neo mellow', 'pop', 'pop rock']",topic_2,Pop,NaN
16,3110,0.8770,['Ed Sheeran'],0.464,252760,0.321,0,5TvFfDlVoUWZvfqrhTJzD7,0.000000,2,...,0.0418,93.528,0.306,2014,Tell me that you'll turn down the man Who asks...,Ed Sheeran,"['pop', 'uk pop']",topic_2,Pop,https://p.scdn.co/mp3-preview/daa8679253ba2062...
17,3133,0.9030,['The Lettermen'],0.500,124533,0.290,0,39xy1RYqpC3XjqPq6XrT7c,0.000059,0,...,0.0308,116.556,0.308,1962,There is someone walking behind you.. Turn aro...,The Lettermen,"['adult standards', 'brill building pop', 'bub...",topic_2,Pop,NaN
25,4076,0.8580,['Barry Manilow'],0.463,277093,0.262,0,14PPUNB4JOzznDNM1w6q7N,0.000002,2,...,0.0496,117.055,0.251,1974,She's a great little housewife Though sometime...,Barry Manilow,"['adult standards', 'bubblegum pop', 'disco', ...",topic_2,Pop,https://p.scdn.co/mp3-preview/c0b2ee1758adfca1...
26,4204,0.0237,['Gladys Knight & The Pips'],0.379,236693,0.326,0,1GdNj9mq8Ix1U40Ppcv2Cr,0.000193,9,...,0.0296,173.528,0.262,1968,The look of love Is in your eyes A look your s...,Gladys Knight & The Pips,"['brill building pop', 'classic soul', 'disco'...",topic_2,Pop,NaN
29,5100,0.3180,['Kenny Loggins'],0.384,281333,0.287,0,6bkOMcBuSjl2cftYa2DrFj,0.000223,5,...,0.0385,105.085,0.349,1977,Home for the holidays I believe I've missed ea...,Kenny Loggins,"['album rock', 'mellow gold', 'new wave pop', ...",topic_2,Pop,https://p.scdn.co/mp3-preview/6e4a53df67943f56...
